In [2]:
english::Vector{String} = ["A", "B", "C", "D", "E", "I"];
greek::Vector{String} = ["α", "β", "γ", "δ", "ϵ", "ϕ"];
alphabets::Vector{String} = [];

append!(alphabets, english)
append!(alphabets, greek)

12-element Vector{String}:
 "A"
 "B"
 "C"
 "D"
 "E"
 "I"
 "α"
 "β"
 "γ"
 "δ"
 "ϵ"
 "ϕ"

### Define the Probability Mass Function of each alphabet

In [3]:
P = Dict();
for alphabet in alphabets
    if string(alphabet) ∈ greek
        P[alphabet] = 1 // 12;
    elseif string(alphabet) ∈ ["A", "E", "I"]
        P[alphabet] = 1 // 9;
    else
        P[alphabet] = 1 // 18;
    end
end

P

Dict{Any, Any} with 12 entries:
  "C" => 1//18
  "ϵ" => 1//12
  "δ" => 1//12
  "B" => 1//18
  "A" => 1//9
  "ϕ" => 1//12
  "D" => 1//18
  "α" => 1//12
  "E" => 1//9
  "γ" => 1//12
  "I" => 1//9
  "β" => 1//12

In [4]:
include("./block_code.jl")
include("./tools.jl")

# Sort Symbols and Probabilities & Create Block Codes
sorted_probabilities, sorted_symbols, P_sorted_dict = sort_prob(P);
block2_codes, prob_block2_codes = create_block_coding(sorted_symbols, sorted_probabilities, 2);
block3_codes, prob_block3_codes = create_block_coding(sorted_symbols, sorted_probabilities, 3);

prob_block2_codes, block2_codes, P2_sorted_dict = sort_prob(prob_block2_codes, block2_codes);
prob_block3_codes, block3_codes, P3_sorted_dict = sort_prob(prob_block3_codes, block3_codes);

In [5]:
include("./huffman.jl")

huffman_tree = construct_huffman_tree(P_sorted_dict)
huffman2_tree = construct_huffman_tree(P2_sorted_dict)
huffman3_tree = construct_huffman_tree(P3_sorted_dict)

encoder1 = Dict();
encoder2 = Dict();
encoder3 = Dict();

build_encoder(huffman_tree, "", encoder1);
build_encoder(huffman2_tree, "", encoder2);
build_encoder(huffman3_tree, "", encoder3);

In [6]:
function report_expected_len(encoder, P, n)
    len = expected_length(encoder, P);
    len = float(len)
    println("Huffman Encoder with block length $n has expected length $(len) bits")
end

report_expected_len(encoder1, P_sorted_dict, 1)
report_expected_len(encoder2, P2_sorted_dict, 2)
report_expected_len(encoder3, P3_sorted_dict, 3)

Huffman Encoder with block length 1 has expected length 3.5833333333333335 bits
Huffman Encoder with block length 2 has expected length 7.114197530864198 bits
Huffman Encoder with block length 3 has expected length 10.664566186556927 bits


In [7]:
# mcmillan_inequality(encoder1)
# mcmillan_inequality(encoder2)
# mcmillan_inequality(encoder3)

In [8]:
language_dict = Dict();
for alphabet in alphabets
    if alphabet ∈ english
        language_dict[alphabet] = "e";
    elseif alphabet ∈ greek
        language_dict[alphabet] = "g";
    end
end

In [9]:
include("./side_information_table.jl")
side1_codebook = create_side_information_codebook(sorted_symbols, language_dict)
side2_codebook = create_side_information_codebook(block2_codes, language_dict)
side3_codebook = create_side_information_codebook(block3_codes, language_dict)

Dict{Any, Any} with 1728 entries:
  "DBϵ" => "eeg"
  "IϕC" => "ege"
  "DβI" => "ege"
  "δαϕ" => "ggg"
  "Bββ" => "egg"
  "αED" => "gee"
  "DIβ" => "eeg"
  "DCα" => "eeg"
  "δϵα" => "ggg"
  "EBD" => "eee"
  "IEB" => "eee"
  "βCϵ" => "geg"
  "BEI" => "eee"
  "IBB" => "eee"
  "ϕαD" => "gge"
  "DCγ" => "eeg"
  "ϵβC" => "gge"
  "δγδ" => "ggg"
  "βαβ" => "ggg"
  ⋮     => ⋮

In [10]:
side_block1, side_prob1 = side_information_table(sorted_symbols, sorted_probabilities, side1_codebook)
side_block2, side_prob2 = side_information_table(block2_codes, prob_block2_codes, side2_codebook)
side_block3, side_prob3 = side_information_table(block3_codes, prob_block3_codes, side3_codebook)

(Dict{String, Vector{Any}}("gee" => ["ϵCC", "δCC", "ϕCC", "αCC", "γCC", "βCC", "ϵBC", "δBC", "ϕBC", "αBC"  …  "ϕEI", "αEI", "γEI", "βEI", "ϵII", "δII", "ϕII", "αII", "γII", "βII"], "eeg" => ["CCϵ", "BCϵ", "DCϵ", "CBϵ", "BBϵ", "DBϵ", "CDϵ", "BDϵ", "DDϵ", "CCδ"  …  "IIγ", "AAβ", "EAβ", "IAβ", "AEβ", "EEβ", "IEβ", "AIβ", "EIβ", "IIβ"], "gge" => ["ϵϵC", "δϵC", "ϕϵC", "αϵC", "γϵC", "βϵC", "ϵδC", "δδC", "ϕδC", "αδC"  …  "ϕγI", "αγI", "γγI", "βγI", "ϵβI", "δβI", "ϕβI", "αβI", "γβI", "ββI"], "ggg" => ["ϵϵϵ", "δϵϵ", "ϕϵϵ", "αϵϵ", "γϵϵ", "βϵϵ", "ϵδϵ", "δδϵ", "ϕδϵ", "αδϵ"  …  "ϕγβ", "αγβ", "γγβ", "βγβ", "ϵββ", "δββ", "ϕββ", "αββ", "γββ", "βββ"], "ege" => ["CϵC", "BϵC", "DϵC", "CδC", "BδC", "DδC", "CϕC", "BϕC", "DϕC", "CαC"  …  "IϕI", "AαI", "EαI", "IαI", "AγI", "EγI", "IγI", "AβI", "EβI", "IβI"], "geg" => ["ϵCϵ", "δCϵ", "ϕCϵ", "αCϵ", "γCϵ", "βCϵ", "ϵBϵ", "δBϵ", "ϕBϵ", "αBϵ"  …  "ϕEβ", "αEβ", "γEβ", "βEβ", "ϵIβ", "δIβ", "ϕIβ", "αIβ", "γIβ", "βIβ"], "eee" => ["CCC", "BCC", "DCC", "CBC", "BBC", "DBC

In [17]:
#=
side_2_block    : Dictionary mapping
                    Side information block symbols => Block Symbols
side_2_block    : Dictionary mapping 
                    Side information block symbols => Block Symbols' Conditional Probablity

Output:
unique_trees    : Dictionary mapping
                    Side information block symbols => Huffman Tree
unique_encoders : Dictionary mapping
                    Side information block symbols => Huffman Encoder given Side Information
=#
function encode_w_side_info(side_2_block, side_2_prob)
    unique_trees = Dict();
    unique_encoders = Dict(); 
    for (side_info, conditional_probabilities) in side_2_prob
        conditioned_symbol = side_2_block[side_info];
        ___, ____, P_dict = sort_prob(conditional_probabilities, conditioned_symbol);
        tree = construct_huffman_tree(P_dict);
        encoder = Dict();
        build_encoder(tree, "", encoder)

        unique_trees[side_info] = tree;
        unique_encoders[side_info] = encoder;
    end
    return unique_trees, unique_encoders
end

side_tree1, side_encoder1 = encode_w_side_info(side_block1, side_prob1)
side_tree2, side_encoder2 = encode_w_side_info(side_block2, side_prob2)
side_tree3, side_encoder3 = encode_w_side_info(side_block3, side_prob3)

(Dict{Any, Any}("gee" => huffman_node{String, Rational{Int64}}("", 1//1, huffman_node{String, Rational{Int64}}("", 115//243, huffman_node{String, Rational{Int64}}("", 17//81, huffman_node{String, Rational{Int64}}("", 19//243, huffman_node{String, Rational{Int64}}("", 8//243, huffman_node{String, Rational{Int64}}("", 4//243, huffman_node{String, Rational{Int64}}("", 2//243, huffman_node{String, Rational{Int64}}("", 1//243, huffman_node{String, Rational{Int64}}("ϵBB", 1//486, nothing, nothing), huffman_node{String, Rational{Int64}}("δBC", 1//486, nothing, nothing)), huffman_node{String, Rational{Int64}}("", 1//243, huffman_node{String, Rational{Int64}}("γBB", 1//486, nothing, nothing), huffman_node{String, Rational{Int64}}("βDB", 1//486, nothing, nothing))), huffman_node{String, Rational{Int64}}("", 2//243, huffman_node{String, Rational{Int64}}("", 1//243, huffman_node{String, Rational{Int64}}("ϕBC", 1//486, nothing, nothing), huffman_node{String, Rational{Int64}}("γDD", 1//486, nothing,

In [26]:
#=
side_2_encoder : Dictionary mapping
                    Side information block symbols => Codeblock Encoder given Side Information
=#
function unroll_side_info_encoders(side_2_encoder)
    encoder_side_info = Dict();
    for (side_info, encoder) in side_2_encoder
        merge!(encoder_side_info, encoder)
    end

    return encoder_side_info
end

unroll_side_info_encoders (generic function with 1 method)

In [29]:
u_side_encoder1 = unroll_side_info_encoders(side_encoder1)
u_side_encoder2 = unroll_side_info_encoders(side_encoder2)
u_side_encoder3 = unroll_side_info_encoders(side_encoder3)

report_expected_len(u_side_encoder1, P_sorted_dict, 1)
report_expected_len(u_side_encoder2, P2_sorted_dict, 2)
report_expected_len(u_side_encoder3, P3_sorted_dict, 3)

Huffman Encoder with block length 1 has expected length 2.611111111111111 bits
Huffman Encoder with block length 2 has expected length 5.151234567901234 bits
Huffman Encoder with block length 3 has expected length 7.688614540466392 bits


In [23]:
decode_huffman(side_tree3["ege"], "0001101")

Going left
Going left
Going left
Going Right
Going Right
Going left
Going Right
Successfuly Decoded: AδA


"AδA\0"